- A web scrapper tool to fetch current index price and Financials of the company from the Google Finance 
- Web scrapping with the BeautifulSoup
- At the end of this notebook, we save data in a csv file for further analysis

In [57]:
#import libraries
import urllib.request
from bs4 import BeautifulSoup

Let's fetch the company information and prices of 10 companies listed in the NASDAQ

In [58]:
stocks=["AAPL","MSFT","GOOGL","TSLA","AMZN","NVDA","META","ASML","AVGO","CSCO"]

In [59]:
def get_soup(stock="TSLA"):
    url = f"https://www.google.com/finance/quote/{stock}:NASDAQ"
    page = urllib.request.urlopen(url=url)
    soup = BeautifulSoup(page, 'html.parser')
    return soup

Get the current index price, financials and other information about the company

The website has financials from the FY Sep 2022 

In [68]:
def info_and_financials(soup):
    """ This function will return a dict that contrains the company name, 
    financials and other info available on the Google Finance website """
    
    
    # Store info in a dict
    trading_info={}
    keys=[]
    values=[]

    # save the date and time of fetching the data
    from datetime import date
    trading_info["date_fetched"] = date.today()
    
    import time
    curr_time = time.strftime("%H:%M:%S",time.gmtime())
    trading_info["time_fetched (UTC)"]=curr_time

    # save current price
    price = soup.find("div",class_="YMlKec fxKbKc").text
    trading_info["current_index_price"] = price

    # company info 
    for tag in soup.find_all("div", class_="mfs7Fc"):
        keys.append(tag.text)
    for tag in soup.find_all("div",class_="P6K39c"):
        if tag.text=='—' or tag.text=="-":
            values.append("")
        else:
            values.append(tag.text)



    # Date at which the latest quaterly financials were reported        
    temp=0
    for x in soup.find_all("th", class_="yNnsfe"):
        temp+=1
        if temp==2:
            trading_info["FinReported"] = f"{x.text}".replace("info"," ").split()[-1].replace(".","")
            break
        
    # Financials from the last finanical year
    for tag in soup.find_all("div",class_="rsPbEe"):
        keys.append(tag.text)
    for tag in soup.find_all("td",class_="QXDnM"):
        values.append(tag.text)

    # Y/Y change in %
    for tag in soup.find_all("div",class_="rsPbEe"):
        keys.append(tag.text + " Y/Y change")
    for tag in soup.find_all("td", class_="gEUVJe"):
        if tag.text=='—':
            values.append("")
        else:
            values.append(tag.text)

    for i in range(len(keys)):
        trading_info[keys[i]]=values[i]
    
    return trading_info

In [69]:
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

In [70]:
dicts={}
for stock in stocks:
    
    soup = get_soup(stock=stock)
    dicts[stock]=info_and_financials(soup=soup)


In [71]:
df=pd.DataFrame(dicts)
print(df.shape)

(58, 10)


In [72]:
df=df.transpose()
print(df.shape)

(10, 58)


In [73]:
Companies=df.index.to_list()
df.insert(0,"Company",Companies)
df.index=np.arange(len(df))
df

,Company,date_fetched,time_fetched (UTC),current_index_price,FinReported,Previous close,Day range,Year range,Market cap,Avg Volume,P/E ratio,Dividend yield,Primary exchange,CDP Climate Change Score,CEO,Founded,Headquarters,Website,Employees,Revenue,Operating expense,Net income,Net profit margin,Earnings per share,EBITDA,Effective tax rate,Cash and short-term investments,Total assets,Total liabilities,Total equity,Shares outstanding,Price to book,Return on assets,Return on capital,Cash from operations,Cash from investing,Cash from financing,Net change in cash,Free cash flow,Revenue Y/Y change,Operating expense Y/Y change,Net income Y/Y change,Net profit margin Y/Y change,Earnings per share Y/Y change,EBITDA Y/Y change,Effective tax rate Y/Y change,Cash and short-term investments Y/Y change,Total assets Y/Y change,Total liabilities Y/Y change,Total equity Y/Y change,Shares outstanding Y/Y change,Price to book Y/Y change,Return on assets Y/Y change,Return on capital Y/Y change,Cash from operations Y/Y change,Cash from investing Y/Y change,Cash from financing Y/Y change,Net change in cash Y/Y change,Free cash flow Y/Y change
0,AAPL,2022-12-29,18:32:14,$129.46,10/27/22,$126.04,$127.73 - $130.48,$125.87 - $182.94,2.06T USD,76.69M,21.19,0.71%,NASDAQ,A-,Tim Cook,"Apr 1, 1976","Cupertino, CaliforniaUnited States",apple.com,"164,000",90.15B,13.20B,20.72B,22.99,1.29,27.76B,15.96%,48.30B,352.76B,302.08B,50.67B,15.91B,39.64,18.06%,34.48%,24.13B,-1.22B,-26.79B,-3.88B,20.18B,8.14%,15.92%,0.83%,-6.73%,,3.68%,,-22.89%,0.50%,4.92%,,,,,,19.44%,-245.75%,-31.46%,-694.79%,51.61%
1,MSFT,2022-12-29,18:32:18,$240.38,10/25/22,$234.53,$235.65 - $241.14,$213.43 - $343.13,1.79T USD,27.37M,25.92,1.13%,NASDAQ,A,Satya Nadella,"Apr 4, 1975","Redmond, WashingtonUnited States",microsoft.com,"221,000",50.12B,13.15B,17.56B,35.03,2.35,24.31B,18.62%,107.24B,359.78B,186.22B,173.57B,7.45B,10.07,14.85%,21.71%,23.20B,-3.13B,-10.88B,8.95B,15.02B,10.60%,15.04%,-14.38%,-22.59%,,3.66%,,-17.87%,7.26%,1.51%,,,,,,-5.47%,3.63%,33.13%,81.20%,-18.12%
2,GOOGL,2022-12-29,18:32:22,$88.31,10/25/22,$86.02,$86.61 - $88.81,$83.34 - $151.55,1.15T USD,28.14M,17.90,,NASDAQ,A-,Sundar Pichai,"Oct 2, 2015","Mountain View, CaliforniaUnited States",abc.xyz,"186,779",69.09B,20.80B,13.91B,20.13,1.06,21.18B,14.31%,116.26B,358.26B,104.63B,253.63B,12.94B,4.40,12.01%,15.10%,23.35B,-833.00M,-18.10B,4.05B,14.25B,6.10%,26.31%,-26.54%,-30.78%,,-12.96%,,-18.13%,3.12%,1.74%,,,,,,-8.56%,91.71%,-18.64%,"4,448.31%",-16.47%
3,TSLA,2022-12-29,18:32:26,$118.86,10/19/22,$112.71,$117.50 - $123.57,$108.24 - $402.67,373.01B USD,116.00M,36.68,,NASDAQ,NaN,Elon Musk,"Jul 1, 2003",NaN,tesla.com,"99,290",21.45B,1.69B,3.29B,15.34,1.05,4.64B,8.39%,21.11B,74.43B,33.30B,41.12B,3.16B,8.93,12.90%,20.19%,5.10B,-2.79B,-712.00M,1.26B,2.06B,55.95%,2.29%,103.46%,30.44%,,67.96%,,31.14%,28.69%,13.50%,,,,,,62.06%,-50.46%,48.44%,"1,063.36%",27.47%
4,AMZN,2022-12-29,18:32:30,$83.63,10/27/22,$81.82,$82.55 - $84.11,$81.69 - $171.40,853.63B USD,75.69M,77.28,,NASDAQ,NaN,Andy Jassy,"Jul 5, 1994","Seattle, WashingtonUnited States",amazon.com,"1,544,000",127.10B,54.31B,2.87B,2.26,0.28,12.73B,2.35%,58.66B,428.36B,290.87B,137.49B,10.20B,6.07,1.49%,2.14%,11.40B,-15.61B,3.02B,-2.52B,-2.40B,14.70%,26.21%,-9.00%,-20.70%,,-7.76%,,-25.73%,12.02%,11.09%,,,,,,55.94%,-5.26%,208.65%,75.96%,23.81%
5,NVDA,2022-12-29,18:32:36,$145.80,11/16/22,$140.36,$142.27 - $146.80,$108.13 - $307.11,359.11B USD,44.40M,62.10,0.11%,NASDAQ,B,Jensen Huang,Apr 1993,"Santa Clara, CaliforniaUnited States",nvidia.com,"22,473",5.93B,2.58B,680.00M,11.47,0.58,1.01B,-10.93%,13.14B,40.49B,19.14B,21.35B,2.46B,16.23,3.58%,4.36%,392.00M,3.15B,-3.75B,-213.00M,528.00M,-16.50%,31.43%,-72.40%,-66.94%,,-66.08%,,-31.89%,-0.35%,13.69%,,,,,,-74.19%,170.92%,-164.30%,95.09%,-74.44%
6,META,2022-12-29,18:32:40,$119.84,10/26/22,$115.62,$115.77 - $120.17,$88.09 - $347.23,317.84B USD,31.36M,11.43,,NASDAQ,NaN,Mark Zuckerberg,Feb 2004,NaN,NaN,"87,314",27.71B,15.95B,4.40B,15.86,1.64,8.25B,21.18%,41.78B,1

In [77]:
from datetime import date
df.to_csv(f"./Data/stock_data_GF_{date.today()}.csv")